In [401]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model 
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate, Conv2D, MaxPool2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pyperclip

In [471]:
# 불러오기
result = pd.read_csv("data/돈까스_R01.csv")

In [472]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category      1999 non-null   object 
 1   ID            1999 non-null   object 
 2   Contents      1999 non-null   object 
 3   Tags          1790 non-null   object 
 4   Tags_sum      1999 non-null   int64  
 5   Tags_count    1999 non-null   int64  
 6   Tags_rate     1999 non-null   float64
 7   Likes         1999 non-null   int64  
 8   Scaled_likes  1999 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 140.7+ KB


In [473]:
result["Tags"] = result["Tags"].fillna("")
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category      1999 non-null   object 
 1   ID            1999 non-null   object 
 2   Contents      1999 non-null   object 
 3   Tags          1999 non-null   object 
 4   Tags_sum      1999 non-null   int64  
 5   Tags_count    1999 non-null   int64  
 6   Tags_rate     1999 non-null   float64
 7   Likes         1999 non-null   int64  
 8   Scaled_likes  1999 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 140.7+ KB


In [323]:
result["Contents"] = result["Contents"].fillna(" ")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        2000 non-null   object
 1   Contents  2000 non-null   object
 2   Tags      2000 non-null   object
 3   Like      1998 non-null   object
dtypes: object(4)
memory usage: 78.1+ KB


In [320]:
result.columns = ["ID", "Contents","Tags","Like"]
result

,ID,Contents,Tags,Like
0,mae_chelin,.서울 5대 #떡볶이 중에 한곳인 #현선이네 20년전 포장마차에서 시작해서 많은 지...,"['#떡볶이', '#현선이네', '#현선이네떡볶이', '#분식', '#생맥주', '...",629
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 🥟솔직히 여기는 이거 먹으러 가는거죠#이...,"['#이수', '#애플하우스', '#무침만두', '#무침군만두', '#만두', '#...",1053
2,biteofyommy,"대전 디저트 간판이 성심당이라면, 떡볶이는 바로그집입니댜. (( #바로그집 )) #...","['#바로그집', '#저장필수', '#대전데이트코스', '#바로그집', '#요미_대전']",1801
3,g.yoooong_place,친구 ’수진‘을 태그하면 꼭 사줘야 하는 곳이 있습니다.경기 성남시 수진역 근처에 ...,"['#수진분식수진분식은', '#떡볶이그', '#김말이', '#계란기본으로', '#굥...",1403
4,mukpresident,🔊 쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다.오늘 저희가 회담을 나눌 곳...,"['#홍반장떡볶이', '#먹통령_신림', '#신림맛집', '#신림역맛집', '#서울...",2633
...,...,...,...,...
1995,yulm___jinhyn,2022.01.08 메뉴: 두끼떡볶이🥘-두끼를 좋아하지만 #떡볶이한판은 #다못먹고날...,"['#떡볶이한판은', '#다못먹고날', '#어묵꼬치10개를', '#해맑게먹는상당히'...",15
1996,i_ate_it__,"금촌 맛집#ok분식 엄청 달달쓰한 덕복희.,. 이날은 떡이 덜익어서 턱이 아팠음ㅠ🥹",['#ok분식'],17
1997,actresshanna,📌 밀키트 5종 공구 오픈 했어요🛒👆🏻프로필에 링크로 가시면 구매 가능하세요하루에 ...,"['#사대부집곳간', '#공동구매', '#공구', '#밀키트', '#집밥', '#맛...",_leelee님 여러 명이 좋아합니
1998,yulm___jinhyn,2022.01.07 메뉴: 차돌불떡🥘-배고파서 정신없이 먹다가 찍은건 #볶음밥뿐불떡...,"['#볶음밥뿐불떡은', '#불고기일까', '#떡볶이일까난', '#그럼떡볶이지-', ...",8


In [321]:
result["Contents"] = result["Contents"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
result["Contents"]

0        서울 5대  떡볶이 중에 한곳인  현선이네 20년전 포장마차에서 시작해서 많은 지...
1       냉장고에 쟁여두고 먹고싶은 빨간 무침군만두  솔직히 여기는 이거 먹으러 가는거죠 이...
2       대전 디저트 간판이 성심당이라면  떡볶이는 바로그집입니댜      바로그집     ...
3       친구  수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...
4         쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...
                              ...                        
1995    2022 01 08 메뉴  두끼떡볶이  두끼를 좋아하지만  떡볶이한판은  다못먹고날...
1996       금촌 맛집 ok분식 엄청 달달쓰한 덕복희    이날은 떡이 덜익어서 턱이 아팠음ㅠ 
1997      밀키트 5종 공구 오픈 했어요   프로필에 링크로 가시면 구매 가능하세요하루에 ...
1998    2022 01 07 메뉴  차돌불떡  배고파서 정신없이 먹다가 찍은건  볶음밥뿐불떡...
1999    서민국수 대표 인기메뉴중 하나인 즉석떡볶이  어디서도 먹어볼수 없는 마성의 땡초양념...
Name: Contents, Length: 2000, dtype: object

In [327]:
result.iloc[1806,3] = 0
result.iloc[1806,3]

0

In [329]:
result["Like_avg"] = result["Like"]
result

,ID,Contents,Tags,Like,Like_avg
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"['#떡볶이', '#현선이네', '#현선이네떡볶이', '#분식', '#생맥주', '...",629,629
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"['#이수', '#애플하우스', '#무침만두', '#무침군만두', '#만두', '#...",1053,1053
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"['#바로그집', '#저장필수', '#대전데이트코스', '#바로그집', '#요미_대전']",1801,1801
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"['#수진분식수진분식은', '#떡볶이그', '#김말이', '#계란기본으로', '#굥...",1403,1403
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"['#홍반장떡볶이', '#먹통령_신림', '#신림맛집', '#신림역맛집', '#서울...",2633,2633
...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"['#떡볶이한판은', '#다못먹고날', '#어묵꼬치10개를', '#해맑게먹는상당히'...",15,15
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,['#ok분식'],17,17
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"['#사대부집곳간', '#공동구매', '#공구', '#밀키트', '#집밥', '#맛...",_leelee님 여러 명이 좋아합니,_leelee님 여러 명이 좋아합니
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"['#볶음밥뿐불떡은', '#불고기일까', '#떡볶이일까난', '#그럼떡볶이지-', ...",8,8


In [330]:
result["Like_mid"] = result["Like"]
result

,ID,Contents,Tags,Like,Like_avg,Like_mid
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"['#떡볶이', '#현선이네', '#현선이네떡볶이', '#분식', '#생맥주', '...",629,629,629
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"['#이수', '#애플하우스', '#무침만두', '#무침군만두', '#만두', '#...",1053,1053,1053
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"['#바로그집', '#저장필수', '#대전데이트코스', '#바로그집', '#요미_대전']",1801,1801,1801
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"['#수진분식수진분식은', '#떡볶이그', '#김말이', '#계란기본으로', '#굥...",1403,1403,1403
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"['#홍반장떡볶이', '#먹통령_신림', '#신림맛집', '#신림역맛집', '#서울...",2633,2633,2633
...,...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"['#떡볶이한판은', '#다못먹고날', '#어묵꼬치10개를', '#해맑게먹는상당히'...",15,15,15
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,['#ok분식'],17,17,17
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"['#사대부집곳간', '#공동구매', '#공구', '#밀키트', '#집밥', '#맛...",_leelee님 여러 명이 좋아합니,_leelee님 여러 명이 좋아합니,_leelee님 여러 명이 좋아합니
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"['#볶음밥뿐불떡은', '#불고기일까', '#떡볶이일까난', '#그럼떡볶이지-', ...",8,8,8


In [331]:
# 평균값
num = result[~(result["Like_avg"].str.contains("좋아합니")|result["Like_avg"].str.contains("눌러보세"))]["Like_avg"]
num = num.astype(int)
num.sum()/len(num)

71.40229226361032

In [332]:
result["Like_avg"].str.contains("좋아합니").value_counts()

False    1765
True      233
Name: Like_avg, dtype: int64

In [333]:
result["Like_avg"].str.contains("눌러보세").value_counts()

False    1976
True       22
Name: Like_avg, dtype: int64

In [334]:
like_str_idx = result[result["Like_avg"].str.contains("좋아합니", na=False)].index
like_0_idx = result[result["Like_avg"].str.contains("눌러보세", na=False)].index

In [335]:
for i in range(len(result)):
    for j in like_str_idx:
        for k in like_0_idx:
            if i==j:
                result["Like_avg"][j]=71.40
            elif i==k:
                result["Like_avg"][k]=0
            else:
                continue

In [336]:
result["Like_avg"].str.contains("좋아합니").value_counts()

False    1743
Name: Like_avg, dtype: int64

In [337]:
result["Like_avg"].str.contains("눌러보세").value_counts()

False    1743
Name: Like_avg, dtype: int64

In [338]:
result["Like_avg"] = result["Like_avg"].astype("int")

In [340]:
#중간값
num = result[~(result["Like_mid"].str.contains("좋아합니")|result["Like_mid"].str.contains("눌러보세"))]["Like_mid"]
num = num.astype(int)
num.median()

22.0

In [341]:
result["Like_mid"].str.contains("좋아합니").value_counts()

False    1765
True      233
Name: Like_mid, dtype: int64

In [342]:
result["Like_mid"].str.contains("눌러보세").value_counts()

False    1976
True       22
Name: Like_mid, dtype: int64

In [343]:
like_str_idx = result[result["Like_mid"].str.contains("좋아합니", na=False)].index
like_0_idx = result[result["Like_mid"].str.contains("눌러보세", na=False)].index

In [344]:
for i in range(len(result)):
    for j in like_str_idx:
        for k in like_0_idx:
            if i==j:
                result["Like_mid"][j]=22
            elif i==k:
                result["Like_mid"][k]=0
            else:
                continue

C:\Users\mhpark\AppData\Local\Temp\ipykernel_31424\131263126.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Like_mid"][k]=0
C:\Users\mhpark\AppData\Local\Temp\ipykernel_31424\131263126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["Like_mid"][j]=22


In [345]:
result["Like_mid"].str.contains("좋아합니").value_counts()

False    1743
Name: Like_mid, dtype: int64

In [346]:
result["Like_mid"].str.contains("눌러보세").value_counts()

False    1743
Name: Like_mid, dtype: int64

In [347]:
result["Like_mid"] = result["Like_mid"].astype("int")

In [348]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        2000 non-null   object
 1   Contents  2000 non-null   object
 2   Tags      2000 non-null   object
 3   Like      2000 non-null   object
 4   Like_avg  2000 non-null   int32 
 5   Like_mid  2000 non-null   int32 
dtypes: int32(2), object(4)
memory usage: 158.3+ KB


In [349]:
# tags # 제외한 특수 기호 제거
result["Tags"] = result["Tags"].str.strip("]['")
result["Tags"] = result["Tags"].str.replace("'", repl=r"", regex=True)

In [350]:
result.head()

,ID,Contents,Tags,Like,Like_avg,Like_mid
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,629,629
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,1053,1053
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,1801,1801
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,1403,1403
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,2633,2633


In [351]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        2000 non-null   object
 1   Contents  2000 non-null   object
 2   Tags      2000 non-null   object
 3   Like      2000 non-null   object
 4   Like_avg  2000 non-null   int32 
 5   Like_mid  2000 non-null   int32 
dtypes: int32(2), object(4)
memory usage: 158.3+ KB


In [352]:
result["Tags"].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
1995    False
1996    False
1997    False
1998    False
1999    False
Name: Tags, Length: 2000, dtype: bool

In [353]:
result = result
result["Tags"] = result["Tags"].fillna("")
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        2000 non-null   object
 1   Contents  2000 non-null   object
 2   Tags      2000 non-null   object
 3   Like      2000 non-null   object
 4   Like_avg  2000 non-null   int32 
 5   Like_mid  2000 non-null   int32 
dtypes: int32(2), object(4)
memory usage: 158.3+ KB


In [354]:
result

,ID,Contents,Tags,Like,Like_avg,Like_mid
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,629,629
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,1053,1053
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,1801,1801
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,1403,1403
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,2633,2633
...,...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"#떡볶이한판은, #다못먹고날, #어묵꼬치10개를, #해맑게먹는상당히, #너, #귀여...",15,15,15
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,#ok분식,17,17,17
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"#사대부집곳간, #공동구매, #공구, #밀키트, #집밥, #맛집, #밀키트추천, #...",_leelee님 여러 명이 좋아합니,71,22
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"#볶음밥뿐불떡은, #불고기일까, #떡볶이일까난, #그럼떡볶이지-, #이대역, #산타...",8,8,8


In [394]:
from tqdm.notebook import tqdm

example_list = ['떡볶이', '먹스타그램', '순대', '맛집', '일상', '분식', '튀김', '떡볶이맛집', '어묵', '맛스타그램'] # 밑에 워드클라우드 전처리과정에서 빈도수 상위10개 추출
Tags_count = [ ]
for i in tqdm(range(len(result))):
    text=[]
    sentence = result["Tags"].str.split("#")[i][1:]
    for s in range(len(sentence)):
        if "," in sentence[s]:
            text.append(sentence[s][:-2])
        else:
            text.append(sentence[s])

    tmp=0
    for j in range(len(example_list)):
        tmp += sum(map(lambda x : 1 if x == example_list[j] else 0, text))
    Tags_count.append(tmp)

result["tags_count"] = Tags_count

  0%|          | 0/2000 [00:00<?, ?it/s]

[5,
 2,
 0,
 0,
 1,
 2,
 5,
 3,
 0,
 3,
 2,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 5,
 3,
 4,
 1,
 0,
 2,
 0,
 3,
 2,
 0,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 3,
 0,
 1,
 1,
 1,
 0,
 3,
 1,
 1,
 3,
 2,
 3,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 4,
 1,
 1,
 0,
 0,
 1,
 1,
 3,
 2,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 2,
 1,
 3,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 3,
 4,
 1,
 1,
 4,
 1,
 0,
 0,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 2,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 1,
 1,
 0,
 3,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 0,
 4,
 1,
 2,
 2,
 4,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 3,
 1,
 3,
 1,
 1,
 1,
 1,
 4,
 1,
 5,
 1,
 4,
 0,
 0,
 2,
 1,
 1,
 2,
 1,
 2,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 3,
 0,
 0,
 4,
 2,
 3,
 1,
 1,
 3,
 4,
 0,
 1,
 0,
 2,
 4,
 4,
 2,
 4,
 4,
 1,
 0,
 3,
 1,
 2,
 2,
 0,
 0,
 4,
 0,
 2,
 0,
 4,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 5,
 1,
 1,
 1,
 1,
 1,
 5,
 3,
 3,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 1,
 1,
 2,
 3,
 2,
 2,
 5,
 4,
 0,
 1,
 5,


In [369]:
result

,ID,Contents,Tags,Like,Like_avg,Like_mid,tags_count
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,629,629,14
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,1053,1053,6
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,1801,1801,0
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,1403,1403,2
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,2633,2633,15
...,...,...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"#떡볶이한판은, #다못먹고날, #어묵꼬치10개를, #해맑게먹는상당히, #너, #귀여...",15,15,15,11
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,#ok분식,17,17,17,1
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"#사대부집곳간, #공동구매, #공구, #밀키트, #집밥, #맛집, #밀키트추천, #...",_leelee님 여러 명이 좋아합니,71,22,6
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"#볶음밥뿐불떡은, #불고기일까, #떡볶이일까난, #그럼떡볶이지-, #이대역, #산타...",8,8,8,8


In [370]:
tag_sum = [ ]
for i in range(len(result)):
    if len(result["Tags"].str.split("#")[1]) >= 1:
        tag_sum.append(len(result["Tags"].str.split("#")[i][1:]))
        
result["tag_sum"] = tag_sum

In [371]:
result["Tags"] = result["Tags"].fillna("")

In [372]:
result["Tags"] = result["Tags"].astype("str")

In [373]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          2000 non-null   object
 1   Contents    2000 non-null   object
 2   Tags        2000 non-null   object
 3   Like        2000 non-null   object
 4   Like_avg    2000 non-null   int32 
 5   Like_mid    2000 non-null   int32 
 6   tags_count  2000 non-null   int64 
 7   tag_sum     2000 non-null   int64 
dtypes: int32(2), int64(2), object(4)
memory usage: 189.5+ KB


In [374]:
result["tag_rate"] = ((result["tags_count"] / result["tag_sum"]) * 100) .round(2)
result.head()

,ID,Contents,Tags,Like,Like_avg,Like_mid,tags_count,tag_sum,tag_rate
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,629,629,14,23,60.87
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,1053,1053,6,11,54.55
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,1801,1801,0,5,0.00
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,1403,1403,2,7,28.57
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,2633,2633,15,11,136.36


In [375]:
result

,ID,Contents,Tags,Like,Like_avg,Like_mid,tags_count,tag_sum,tag_rate
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,629,629,14,23,60.87
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,1053,1053,6,11,54.55
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,1801,1801,0,5,0.00
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,1403,1403,2,7,28.57
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,2633,2633,15,11,136.36
...,...,...,...,...,...,...,...,...,...
1995,yulm___jinhyn,2022 01 08 메뉴 두끼떡볶이 두끼를 좋아하지만 떡볶이한판은 다못먹고날...,"#떡볶이한판은, #다못먹고날, #어묵꼬치10개를, #해맑게먹는상당히, #너, #귀여...",15,15,15,11,19,57.89
1996,i_ate_it__,금촌 맛집 ok분식 엄청 달달쓰한 덕복희 이날은 떡이 덜익어서 턱이 아팠음ㅠ,#ok분식,17,17,17,1,1,100.00
1997,actresshanna,밀키트 5종 공구 오픈 했어요 프로필에 링크로 가시면 구매 가능하세요하루에 ...,"#사대부집곳간, #공동구매, #공구, #밀키트, #집밥, #맛집, #밀키트추천, #...",_leelee님 여러 명이 좋아합니,71,22,6,30,20.00
1998,yulm___jinhyn,2022 01 07 메뉴 차돌불떡 배고파서 정신없이 먹다가 찍은건 볶음밥뿐불떡...,"#볶음밥뿐불떡은, #불고기일까, #떡볶이일까난, #그럼떡볶이지-, #이대역, #산타...",8,8,8,8,15,53.33


In [376]:
result["tag_rate"] = result["tag_rate"].fillna(0)
result["tag_rate"]

0        60.87
1        54.55
2         0.00
3        28.57
4       136.36
         ...  
1995     57.89
1996    100.00
1997     20.00
1998     53.33
1999     19.23
Name: tag_rate, Length: 2000, dtype: float64

In [377]:
result['Like_avg_normal1'] = result['Like_avg'] / abs(result['Like_avg'].max())
result['Like_avg_normal1'].head()

0    0.073878
1    0.123679
2    0.211534
3    0.164787
4    0.309255
Name: Like_avg_normal1, dtype: float64

In [378]:
min_x = result['Like_avg'] - result['Like_avg'].min()
min_max = result['Like_avg'].max() - result['Like_avg'].min()

result['Like_avg_normal2'] = min_x / min_max
result['Like_avg_normal2'].head()

0    0.073878
1    0.123679
2    0.211534
3    0.164787
4    0.309255
Name: Like_avg_normal2, dtype: float64

In [379]:
result['Like_mid_normal1'] = result['Like_mid'] / abs(result['Like_mid'].max())
result['Like_mid_normal1'].head()

0    0.073878
1    0.123679
2    0.211534
3    0.164787
4    0.309255
Name: Like_mid_normal1, dtype: float64

In [380]:
min_x = result['Like_mid'] - result['Like_mid'].min()
min_max = result['Like_mid'].max() - result['Like_mid'].min()

result['Like_mid_normal2'] = min_x / min_max
result['Like_mid_normal2'].head()

0    0.073878
1    0.123679
2    0.211534
3    0.164787
4    0.309255
Name: Like_mid_normal2, dtype: float64

In [381]:
result.head()

,ID,Contents,Tags,Like,Like_avg,Like_mid,tags_count,tag_sum,tag_rate,Like_avg_normal1,Like_avg_normal2,Like_mid_normal1,Like_mid_normal2
0,mae_chelin,서울 5대 떡볶이 중에 한곳인 현선이네 20년전 포장마차에서 시작해서 많은 지...,"#떡볶이, #현선이네, #현선이네떡볶이, #분식, #생맥주, #현선이네본점, #즉석...",629,629,629,14,23,60.87,0.073878,0.073878,0.073878,0.073878
1,_yamiyomi_,냉장고에 쟁여두고 먹고싶은 빨간 무침군만두 솔직히 여기는 이거 먹으러 가는거죠 이...,"#이수, #애플하우스, #무침만두, #무침군만두, #만두, #양념만두, #분식, #...",1053,1053,1053,6,11,54.55,0.123679,0.123679,0.123679,0.123679
2,biteofyommy,대전 디저트 간판이 성심당이라면 떡볶이는 바로그집입니댜 바로그집 ...,"#바로그집, #저장필수, #대전데이트코스, #바로그집, #요미_대전",1801,1801,1801,0,5,0.00,0.211534,0.211534,0.211534,0.211534
3,g.yoooong_place,친구 수진 을 태그하면 꼭 사줘야 하는 곳이 있습니다 경기 성남시 수진역 근처에 ...,"#수진분식수진분식은, #떡볶이그, #김말이, #계란기본으로, #굥플_경기, #굥플_...",1403,1403,1403,2,7,28.57,0.164787,0.164787,0.164787,0.164787
4,mukpresident,쌀떡볶이파 국민 주목친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳...,"#홍반장떡볶이, #먹통령_신림, #신림맛집, #신림역맛집, #서울대입구맛집, #쌀떡...",2633,2633,2633,15,11,136.36,0.309255,0.309255,0.309255,0.309255


In [382]:
# 저장
result.to_csv("data/떡볶이2000-1.csv", encoding="utf-8", index=False)

In [162]:
# 불러오기
result = pd.read_csv("data/돈까스2000-1.csv")

In [163]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Contents          1999 non-null   object 
 1   Like              1999 non-null   object 
 2   ID                1999 non-null   object 
 3   Tags              1790 non-null   object 
 4   Like_avg          1999 non-null   int64  
 5   Like_mid          1999 non-null   int64  
 6   tags_count        1999 non-null   int64  
 7   tag_sum           1999 non-null   int64  
 8   tag_rate          1999 non-null   float64
 9   Like_avg_normal1  1999 non-null   float64
 10  Like_avg_normal2  1999 non-null   float64
 11  Like_mid_normal1  1999 non-null   float64
 12  Like_mid_normal2  1999 non-null   float64
dtypes: float64(5), int64(4), object(4)
memory usage: 203.1+ KB


In [474]:
import time
from tqdm import tqdm_notebook

content_all = ''
for i in tqdm_notebook(range(len(result["Contents"]))):
    content_all = content_all + ' ' + result["Contents"].loc[i]

C:\Users\mhpark\AppData\Local\Temp\ipykernel_31424\1026978283.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(result["Contents"]))):


  0%|          | 0/1999 [00:00<?, ?it/s]

In [475]:
import re
content_all = re.sub('[^ㄱ-ㅣ가-힣 ]',"",content_all)
content_all

' 내 꿈은 돈모아서  돈까스의집 사는것  한입에 모든걸 뺏겨버린 맛이다  괜히 년 전통을 이어온곳이 아니다  일단 에피타이저로 나오는 스프는 아는 맛이 무서운법  위장달래기에 딱이다  사람들로 북적북적한 옛날 감성 제대로 느낄 수 있다  밥으로 하시겠습니까  아니면 빵으로 하시겠습니까  제발 둘다 하자  그리고 빵은 한번 더 추가해야한다  버터향 돈까스 생선까스 함박까지 제대로 즐길 수 있는 정식은 일단 실하다  경양식 돈까스의 정석  고기도 두툼하고 튀김옷은 빠싹 그리고 소스는 존맛  한입 썰어먹으면 끝난다    위치   서울 송파구 삼전로    삼전역 번 출구에서  메뉴   돈까스정식   돈까스맛집  송파맛집  삼전역맛집  빵맛집  돈까스 구름같은 만두를 맛볼 수 있은 곳이에요  너무 맛있어서 음소거 모드로 흡입했어요  건대 성수완당깔끔한 국물과 쫄깃한면발  대왕닭다리의 삼위일체가 미쳤어요  기본 매운맛 가지가 있는데 개인적으로는 매운맛 추천드려요  입 안을 때리는 칼칼한 매운맛이라 취향저격했어요  국물 위에 펼쳐져 있는 구름만두의 식감은 설명이 필요 없어요  숟가락으로 국물과 만두를 함께 떠서 먹어주면 극락가요  밥 말아서 먹으면 양이 배  바질 로제돈까스도 꾸덕한 소스와 바삭한 돈까스의 콜라보가 미쳤어요  소스가 살짝 매콤해서 돈까스 특유의 기름진 맛을 꽉 잡아줘요  만두소가 꽉 차있는 가지튀김도 놓치지마세요  가지 특유의 수분감이 그대로 느껴지는 촉촉한 식감  입 안 가득 들어차는 뚱뚱한 크기에서 오는 만족감이 대박   달달한 칠리 소스랑 잘 어울려요   성수완당건대점 서울 광진구 동일로길 의정부점  경기 의정부시 시민로  층 투어건대   고치돈 좋아하는 국민들 주목 친애하는 국민여러분 먹통령입니다 오늘 저희가 회담을 나눌 곳은  사장님돈까스 입니다  이대에 위치한 이곳 돈까스집은고구마무스 지분율 원탑인 돈까스를 파는 곳입니다 여러분  이게 바로 고구마무스 지분율 위인 돈까스입니다  달콤부들부들한 샛노란 고구마무스와 파삭한 돈까스의 만남은 누가 뭐래

In [476]:
from konlpy.tag import Okt
okt = Okt()
nouns_txt = okt.nouns(content_all)
nouns_txt

['내',
 '꿈',
 '돈',
 '돈까스',
 '집',
 '입',
 '걸',
 '겨',
 '맛',
 '년',
 '전통',
 '곳',
 '일단',
 '에피타이저',
 '스프',
 '맛',
 '법',
 '위장',
 '사람',
 '북적',
 '북적',
 '옛날',
 '감성',
 '제대로',
 '수',
 '밥',
 '빵',
 '제발',
 '하자',
 '빵',
 '한번',
 '더',
 '추가',
 '버터',
 '향',
 '돈까스',
 '생선',
 '까스',
 '함박',
 '제대로',
 '수',
 '정식',
 '일단',
 '양식',
 '돈까스',
 '정석',
 '고기',
 '두툼',
 '튀김',
 '옷',
 '빠싹',
 '소스',
 '존맛',
 '입',
 '위치',
 '서울',
 '송파구',
 '삼전로',
 '전역',
 '번',
 '출구',
 '메뉴',
 '돈까스',
 '정식',
 '돈까스',
 '맛집',
 '송파',
 '맛집',
 '전역',
 '맛집',
 '빵',
 '맛집',
 '돈까스',
 '구름',
 '만두',
 '수',
 '곳',
 '음소거',
 '모드',
 '흡입',
 '건대',
 '수완',
 '국물',
 '쫄깃한면발',
 '대왕',
 '닭다리',
 '삼위일체',
 '기본',
 '맛',
 '가지',
 '개인',
 '맛',
 '추천',
 '입',
 '안',
 '맛',
 '취향',
 '저격',
 '국물',
 '위',
 '구름',
 '만두',
 '식',
 '설명',
 '필요',
 '숟가락',
 '국물',
 '만두',
 '극락',
 '가요',
 '밥',
 '양',
 '배',
 '바질',
 '로',
 '돈까스',
 '덕',
 '소스',
 '바삭',
 '돈까스',
 '콜라보',
 '소스',
 '살짝',
 '매콤',
 '돈까스',
 '특유',
 '맛',
 '꽉',
 '만두소',
 '꽉',
 '가지',
 '튀김',
 '가지',
 '특유',
 '수분',
 '감',
 '그대로',
 '식감',
 '입',
 '안',
 '가득',
 '크기',
 '만족감',
 '대박',
 '달달',


In [477]:
from collections import Counter
count = Counter(nouns_txt)
count
rank_text = count.most_common()
rank_text

[('돈까스', 4430),
 ('맛집', 3215),
 ('그램', 1134),
 ('스타', 928),
 ('카츠', 900),
 ('맛', 479),
 ('치즈', 471),
 ('오늘', 442),
 ('메뉴', 410),
 ('추천', 407),
 ('돈카츠', 387),
 ('점심', 359),
 ('카페', 306),
 ('맞팔', 304),
 ('일상', 295),
 ('돈가스', 292),
 ('집', 288),
 ('소스', 283),
 ('밥', 276),
 ('우동', 268),
 ('등심', 256),
 ('더', 243),
 ('맛스타', 231),
 ('배달', 231),
 ('곳', 225),
 ('고기', 222),
 ('튀김', 222),
 ('저녁', 220),
 ('돈', 219),
 ('수', 219),
 ('맘', 219),
 ('점', 211),
 ('시간', 196),
 ('세트', 196),
 ('양식', 195),
 ('음식', 194),
 ('집밥', 193),
 ('식당', 188),
 ('안동', 187),
 ('카레', 186),
 ('주문', 184),
 ('것', 176),
 ('안심', 175),
 ('먹방', 172),
 ('떡볶이', 165),
 ('데이트', 165),
 ('비', 158),
 ('서울', 157),
 ('부산', 152),
 ('파스타', 151),
 ('소통', 151),
 ('때', 149),
 ('분', 143),
 ('방문', 143),
 ('원', 142),
 ('요리', 141),
 ('여행', 138),
 ('수제', 136),
 ('선팔', 135),
 ('반찬', 134),
 ('까스', 131),
 ('김치', 130),
 ('미사', 123),
 ('샐러드', 122),
 ('생각', 120),
 ('선물', 119),
 ('시', 119),
 ('일', 119),
 ('바삭', 117),
 ('소바', 117),
 ('날', 117),
 ('층', 116),

In [478]:
rank_text

[('돈까스', 4430),
 ('맛집', 3215),
 ('그램', 1134),
 ('스타', 928),
 ('카츠', 900),
 ('맛', 479),
 ('치즈', 471),
 ('오늘', 442),
 ('메뉴', 410),
 ('추천', 407),
 ('돈카츠', 387),
 ('점심', 359),
 ('카페', 306),
 ('맞팔', 304),
 ('일상', 295),
 ('돈가스', 292),
 ('집', 288),
 ('소스', 283),
 ('밥', 276),
 ('우동', 268),
 ('등심', 256),
 ('더', 243),
 ('맛스타', 231),
 ('배달', 231),
 ('곳', 225),
 ('고기', 222),
 ('튀김', 222),
 ('저녁', 220),
 ('돈', 219),
 ('수', 219),
 ('맘', 219),
 ('점', 211),
 ('시간', 196),
 ('세트', 196),
 ('양식', 195),
 ('음식', 194),
 ('집밥', 193),
 ('식당', 188),
 ('안동', 187),
 ('카레', 186),
 ('주문', 184),
 ('것', 176),
 ('안심', 175),
 ('먹방', 172),
 ('떡볶이', 165),
 ('데이트', 165),
 ('비', 158),
 ('서울', 157),
 ('부산', 152),
 ('파스타', 151),
 ('소통', 151),
 ('때', 149),
 ('분', 143),
 ('방문', 143),
 ('원', 142),
 ('요리', 141),
 ('여행', 138),
 ('수제', 136),
 ('선팔', 135),
 ('반찬', 134),
 ('까스', 131),
 ('김치', 130),
 ('미사', 123),
 ('샐러드', 122),
 ('생각', 120),
 ('선물', 119),
 ('시', 119),
 ('일', 119),
 ('바삭', 117),
 ('소바', 117),
 ('날', 117),
 ('층', 116),

In [272]:
print(result["Tags"][0])
print(len(result["Tags"][0]))

#寿司, #鮨, #sushi, #すし, #초밥, #스시, #北の華はやし, #千歳, #千歳グルメ, #千歳カフェ, #新千歳空港, #新千歳空港グルメ, #新千歳, #新千歳空港限定, #札幌グルメ, #札幌観光, #北海道グルメ, #hokkaido, #hokkaidotrip, #sapporo, #寿司スタグラム, #予約困難店, #ミシュラン, #ミシュラン北海道, #インスタグルメアワード2023, #北海道じゃらん, #poroco, #寿司の虎, #虎の日常
254


In [273]:
Tag = []
for i in result["Tags"]:
    ex = i[2:-2].split("', '")
    for j in ex:
        Tag.append(j)
Tag

['司, #鮨, #sushi, #すし, #초밥, #스시, #北の華はやし, #千歳, #千歳グルメ, #千歳カフェ, #新千歳空港, #新千歳空港グルメ, #新千歳, #新千歳空港限定, #札幌グルメ, #札幌観光, #北海道グルメ, #hokkaido, #hokkaidotrip, #sapporo, #寿司スタグラム, #予約困難店, #ミシュラン, #ミシュラン北海道, #インスタグルメアワード2023, #北海道じゃらん, #poroco, #寿司の虎, #虎の',
 '宿焼肉ブルズ, #焼肉ブルズ, #新宿焼肉, #焼肉, #肉寿司, #肉寿司食べ放題, #東京焼肉, #焼肉食べ放題, #東京肉寿司, #雛鮨, #寿司, #寿司食べ放題, #銀座グルメ, #銀座ランチ, #新宿グルメ, #炉端焼き, #おでん食べ放題, #東京おでん, #新宿おでん, #東京食べ放題, #食べ放題, #東京グルメ, #恋文横丁, #渋谷グルメ, #渋谷ランチ, #渋谷居酒屋, #新宿ランチ, #75chacha, #新大久保グルメ, #',
 '자씨, #먹통령_홍대, #홍대맛집, #홍대횟집, #연남동술집, #해물찜, #해물탕, #스시',
 '찬, #제주미묘✨메뉴, #미묘, #제주이자카야, #제주사시미맛집, #제주일식, #제주술집, #제주다이닝, #제주스시, #이도동맛집, #제주초밥, #신제주맛집, #제주공항근처맛집, #제주시맛집, #제주도민맛집, #제주도맛집, #제주맛집탐방, #제주핫플, #제주도여행, #제주가볼만한곳, #제주핫플레이스, #일식, #이자카야, #술집, #제주여행, #스시, #',
 '래여거, #간장게장, #먹교_이대, #이대오마카세, #가성비오마카세, #이대여래여거, #이대맛집, #오마카세, #스시, #오마카세추천, #서울오마카세, #신촌맛집, #이대역맛집, #이화여대맛집, #간장',
 '시렌, #비터팬⠀⠀P.',
 '花車, #鮨あらい, #ウニ王子チャンネル, #ウニ王子, #ウニ王子の食べログ, #鮨, #寿司, #sushi, #YouTube, #グルメ部, #グルメ男子, #港区グルメ, #鮨好きな人と繋がりたい, #ミシュラン, #M

In [274]:
Tag_count = Counter(Tag)
Tag_count

Counter({'司, #鮨, #sushi, #すし, #초밥, #스시, #北の華はやし, #千歳, #千歳グルメ, #千歳カフェ, #新千歳空港, #新千歳空港グルメ, #新千歳, #新千歳空港限定, #札幌グルメ, #札幌観光, #北海道グルメ, #hokkaido, #hokkaidotrip, #sapporo, #寿司スタグラム, #予約困難店, #ミシュラン, #ミシュラン北海道, #インスタグルメアワード2023, #北海道じゃらん, #poroco, #寿司の虎, #虎の': 3,
         '宿焼肉ブルズ, #焼肉ブルズ, #新宿焼肉, #焼肉, #肉寿司, #肉寿司食べ放題, #東京焼肉, #焼肉食べ放題, #東京肉寿司, #雛鮨, #寿司, #寿司食べ放題, #銀座グルメ, #銀座ランチ, #新宿グルメ, #炉端焼き, #おでん食べ放題, #東京おでん, #新宿おでん, #東京食べ放題, #食べ放題, #東京グルメ, #恋文横丁, #渋谷グルメ, #渋谷ランチ, #渋谷居酒屋, #新宿ランチ, #75chacha, #新大久保グルメ, #': 1,
         '자씨, #먹통령_홍대, #홍대맛집, #홍대횟집, #연남동술집, #해물찜, #해물탕, #스시': 1,
         '찬, #제주미묘✨메뉴, #미묘, #제주이자카야, #제주사시미맛집, #제주일식, #제주술집, #제주다이닝, #제주스시, #이도동맛집, #제주초밥, #신제주맛집, #제주공항근처맛집, #제주시맛집, #제주도민맛집, #제주도맛집, #제주맛집탐방, #제주핫플, #제주도여행, #제주가볼만한곳, #제주핫플레이스, #일식, #이자카야, #술집, #제주여행, #스시, #': 1,
         '래여거, #간장게장, #먹교_이대, #이대오마카세, #가성비오마카세, #이대여래여거, #이대맛집, #오마카세, #스시, #오마카세추천, #서울오마카세, #신촌맛집, #이대역맛집, #이화여대맛집, #간장': 1,
         '시렌, #비터팬⠀⠀P.': 1,
         '花車, #鮨あらい, #ウニ王子チャンネル, #ウニ王子, #ウニ王子の食べログ, 

In [275]:
rank_text = Tag_count.most_common()
rank_text

[('밥, #초밥맛집, #대학로초밥, #대학로스시마시따, #대학로데이트, #대학로연극, #대학로스시, #대학로맛집, #대학로카페, #혜화역스시, #혜화역초밥, #혜화역연극, #혜화역맛집, #혜화역맛집, #혜화역데이트, #일상그램, #사주잘보는곳, #타로점, #사주운세, #사주카페, #사주상담, #서울대학교병원, #서울맛집, #스시, #대학로스시마',
  240),
 ('', 226),
 ('운탕, #사시미, #회스타그램, #석화, #초밥, #초밥뷔페, #회전초밥, #스시, #도마29, #일식, #마늘족발, #양념족발, #잡채밥, #고추잡채, #짬짜면, #사천탕수육, #피자스타그램, #치즈피자, #포테이토피자, #시카고피자, #하프앤하프, #쉬림프골드, #화덕피자, #피자빵, #삼겹살데이, #부업',
  134),
 ('운탕, #사시미, #회스타그램, #석화, #초밥, #초밥뷔페, #회전초밥, #스시, #도마29, #일식, #마늘족발, #양념족발, #부업재테크, #제태크부업, #송민영재테크',
  25),
 ('스시, #신부동맛집, #신부동초밥, #신부동데이트코스, #신부동맛집수스시, #동남구초밥, #초밥, #스시, #맛집, #맛집스타그램, #맛집추천, #맛집',
  15),
 ('寿司, #国際通りのれん街, #沖縄, #国際通り, #okinawa, #寿司, #日本酒, #沖縄旅行, #1人飲み, #寿司屋, #居酒屋, #沖縄グルメ, #沖縄寿司, #오키나와스시, #어수사, #우오즈시, #那覇グルメ, #魚寿司国際通りのれん街, #오키나와, #',
  10),
 ('스시바로, #수스시, #신부동맛집, #신부동초밥, #신부동데이트코스, #신부동맛집수스시, #동남구초밥, #초밥, #스시, #맛집, #맛집스타그램, #맛집추천, #맛집',
  9),
 ('갑내기, #매운탕, #사시미, #회스타그램, #석화, #초밥, #초밥뷔페, #회전초밥, #스시, #도마29, #일식, #마늘족발, #양념족발, #잡채밥, #고추잡채, #짬짜면, #사천탕수육, #피자스타그램, #치즈피자, #포테이토피

In [276]:
rank_text = np.array(rank_text)
rank_text = rank_text[0:,0]

In [277]:
rt = rank_text[0:10]
rt

array(['밥, #초밥맛집, #대학로초밥, #대학로스시마시따, #대학로데이트, #대학로연극, #대학로스시, #대학로맛집, #대학로카페, #혜화역스시, #혜화역초밥, #혜화역연극, #혜화역맛집, #혜화역맛집, #혜화역데이트, #일상그램, #사주잘보는곳, #타로점, #사주운세, #사주카페, #사주상담, #서울대학교병원, #서울맛집, #스시, #대학로스시마',
       '',
       '운탕, #사시미, #회스타그램, #석화, #초밥, #초밥뷔페, #회전초밥, #스시, #도마29, #일식, #마늘족발, #양념족발, #잡채밥, #고추잡채, #짬짜면, #사천탕수육, #피자스타그램, #치즈피자, #포테이토피자, #시카고피자, #하프앤하프, #쉬림프골드, #화덕피자, #피자빵, #삼겹살데이, #부업',
       '운탕, #사시미, #회스타그램, #석화, #초밥, #초밥뷔페, #회전초밥, #스시, #도마29, #일식, #마늘족발, #양념족발, #부업재테크, #제태크부업, #송민영재테크',
       '스시, #신부동맛집, #신부동초밥, #신부동데이트코스, #신부동맛집수스시, #동남구초밥, #초밥, #스시, #맛집, #맛집스타그램, #맛집추천, #맛집',
       '寿司, #国際通りのれん街, #沖縄, #国際通り, #okinawa, #寿司, #日本酒, #沖縄旅行, #1人飲み, #寿司屋, #居酒屋, #沖縄グルメ, #沖縄寿司, #오키나와스시, #어수사, #우오즈시, #那覇グルメ, #魚寿司国際通りのれん街, #오키나와, #',
       '스시바로, #수스시, #신부동맛집, #신부동초밥, #신부동데이트코스, #신부동맛집수스시, #동남구초밥, #초밥, #스시, #맛집, #맛집스타그램, #맛집추천, #맛집',
       '갑내기, #매운탕, #사시미, #회스타그램, #석화, #초밥, #초밥뷔페, #회전초밥, #스시, #도마29, #일식, #마늘족발, #양념족발, #잡채밥, #고추잡채, #짬짜면, #사천탕수육, #피자스타그램, #치즈피자, #포테이토피자, #시카고피자

In [278]:
rank = []
for i in rt:
    ex = i[:].split(", ")
    for j in ex:
        rank.append(j)
rank

['밥',
 '#초밥맛집',
 '#대학로초밥',
 '#대학로스시마시따',
 '#대학로데이트',
 '#대학로연극',
 '#대학로스시',
 '#대학로맛집',
 '#대학로카페',
 '#혜화역스시',
 '#혜화역초밥',
 '#혜화역연극',
 '#혜화역맛집',
 '#혜화역맛집',
 '#혜화역데이트',
 '#일상그램',
 '#사주잘보는곳',
 '#타로점',
 '#사주운세',
 '#사주카페',
 '#사주상담',
 '#서울대학교병원',
 '#서울맛집',
 '#스시',
 '#대학로스시마',
 '',
 '운탕',
 '#사시미',
 '#회스타그램',
 '#석화',
 '#초밥',
 '#초밥뷔페',
 '#회전초밥',
 '#스시',
 '#도마29',
 '#일식',
 '#마늘족발',
 '#양념족발',
 '#잡채밥',
 '#고추잡채',
 '#짬짜면',
 '#사천탕수육',
 '#피자스타그램',
 '#치즈피자',
 '#포테이토피자',
 '#시카고피자',
 '#하프앤하프',
 '#쉬림프골드',
 '#화덕피자',
 '#피자빵',
 '#삼겹살데이',
 '#부업',
 '운탕',
 '#사시미',
 '#회스타그램',
 '#석화',
 '#초밥',
 '#초밥뷔페',
 '#회전초밥',
 '#스시',
 '#도마29',
 '#일식',
 '#마늘족발',
 '#양념족발',
 '#부업재테크',
 '#제태크부업',
 '#송민영재테크',
 '스시',
 '#신부동맛집',
 '#신부동초밥',
 '#신부동데이트코스',
 '#신부동맛집수스시',
 '#동남구초밥',
 '#초밥',
 '#스시',
 '#맛집',
 '#맛집스타그램',
 '#맛집추천',
 '#맛집',
 '寿司',
 '#国際通りのれん街',
 '#沖縄',
 '#国際通り',
 '#okinawa',
 '#寿司',
 '#日本酒',
 '#沖縄旅行',
 '#1人飲み',
 '#寿司屋',
 '#居酒屋',
 '#沖縄グルメ',
 '#沖縄寿司',
 '#오키나와스시',
 '#어수사',
 '#우오즈시',
 '#那覇グルメ',
 '#魚寿司国際通りのれん街',
 '#오키나와',
 '#',
 '스시바로

In [479]:
Content = []
for i in result["Contents"]:
    ex = i[2:-2].split(" ")
    for j in ex:
        Content.append(j)
Content

['꿈은',
 '돈모아서',
 '',
 '돈까스의집',
 '사는것',
 '',
 '한입에',
 '모든걸',
 '뺏겨버린',
 '맛이다',
 '',
 '괜히',
 '40년',
 '전통을',
 '이어온곳이',
 '아니다',
 '',
 '일단',
 '에피타이저로',
 '나오는',
 '스프는',
 '아는',
 '맛이',
 '무서운법',
 '',
 '위장달래기에',
 '딱이다',
 '',
 '사람들로',
 '북적북적한',
 '옛날',
 '감성',
 '제대로',
 '느낄',
 '수',
 '있다',
 '',
 '밥으로',
 '하시겠습니까',
 '',
 '아니면',
 '빵으로',
 '하시겠습니까',
 '',
 '제발',
 '둘다',
 '하자',
 '',
 '그리고',
 '빵은',
 '한번',
 '더',
 '추가해야한다',
 '',
 '버터향',
 '돈까스',
 '생선까스',
 '함박까지',
 '제대로',
 '즐길',
 '수',
 '있는',
 '정식은',
 '일단',
 '실하다',
 '',
 '경양식',
 '돈까스의',
 '정석',
 '',
 '고기도',
 '두툼하고',
 '튀김옷은',
 '빠싹',
 '그리고',
 '소스는',
 '존맛',
 '',
 '한입',
 '썰어먹으면',
 '끝난다',
 '',
 '',
 '',
 '위치',
 '',
 '',
 '서울',
 '송파구',
 '삼전로',
 '100',
 '',
 '',
 '삼전역',
 '2번',
 '출구에서382m',
 '',
 '메뉴',
 '',
 '',
 '돈까스정식',
 '11000',
 '',
 '돈까스맛집',
 '',
 '송파맛집',
 '',
 '삼전역맛집',
 '',
 '빵맛집',
 '',
 '돈',
 '같은',
 '만두를',
 '맛볼',
 '수',
 '있은',
 '곳이에요',
 '',
 '너무',
 '맛있어서',
 '음소거',
 '모드로',
 '흡입했어요',
 '',
 '건대',
 '성수완당깔끔한',
 '국물과',
 '쫄깃한면발',
 '',
 '대왕닭다리의',
 '삼위일체가',
 '미쳤어요',
 '',
 '기본'

In [480]:
Content_count = Counter(Content)
Content_count

Counter({'꿈은': 1,
         '돈모아서': 1,
         '': 32225,
         '돈까스의집': 4,
         '사는것': 1,
         '한입에': 3,
         '모든걸': 3,
         '뺏겨버린': 1,
         '맛이다': 6,
         '괜히': 6,
         '40년': 1,
         '전통을': 1,
         '이어온곳이': 1,
         '아니다': 6,
         '일단': 18,
         '에피타이저로': 2,
         '나오는': 34,
         '스프는': 4,
         '아는': 12,
         '맛이': 64,
         '무서운법': 1,
         '위장달래기에': 1,
         '딱이다': 1,
         '사람들로': 1,
         '북적북적한': 1,
         '옛날': 29,
         '감성': 15,
         '제대로': 13,
         '느낄': 17,
         '수': 220,
         '있다': 32,
         '밥으로': 2,
         '하시겠습니까': 4,
         '아니면': 5,
         '빵으로': 3,
         '제발': 3,
         '둘다': 6,
         '하자': 1,
         '그리고': 70,
         '빵은': 2,
         '한번': 28,
         '더': 163,
         '추가해야한다': 1,
         '버터향': 1,
         '돈까스': 1912,
         '생선까스': 35,
         '함박까지': 1,
         '즐길': 23,
         '있는': 131,
         '정식은': 1,
         '실하다': 1,
    

In [481]:
rank_text = Content_count.most_common()
rank_text

[('', 32225),
 ('돈까스', 1912),
 ('맛집', 364),
 ('먹스타그램', 270),
 ('돈까스맛집', 252),
 ('00', 223),
 ('수', 220),
 ('맛스타그램', 180),
 ('돈카츠', 178),
 ('돈', 176),
 ('더', 163),
 ('일상', 150),
 ('점심', 136),
 ('너무', 134),
 ('11', 133),
 ('있는', 131),
 ('30', 131),
 ('잘', 123),
 ('1', 121),
 ('먹스타', 120),
 ('다', 114),
 ('많이', 108),
 ('집밥', 107),
 ('돈가스', 105),
 ('치즈돈까스', 103),
 ('맞팔', 103),
 ('한', 102),
 ('먹고', 102),
 ('정말', 99),
 ('먹방', 96),
 ('진짜', 95),
 ('맛있는', 94),
 ('먹팔', 93),
 ('좋아요', 91),
 ('우동', 90),
 ('이', 88),
 ('또', 87),
 ('맛있게', 86),
 ('서울', 82),
 ('먹을', 78),
 ('15', 78),
 ('함께', 75),
 ('떡볶이', 73),
 ('food', 73),
 ('것', 72),
 ('카츠', 72),
 ('오늘도', 72),
 ('오늘', 71),
 ('그리고', 70),
 ('1층', 67),
 ('바로', 67),
 ('좋은', 66),
 ('맛', 66),
 ('소통', 66),
 ('저녁', 66),
 ('14', 65),
 ('맛이', 64),
 ('경양식', 64),
 ('파스타', 63),
 ('메뉴', 62),
 ('카레', 62),
 ('맛있다', 62),
 ('그', 62),
 ('샐러드', 61),
 ('맛집추천', 61),
 ('17', 60),
 ('까스', 60),
 ('とんかつ', 59),
 ('먹으면', 58),
 ('등심', 58),
 ('2', 58),
 ('직접', 58),
 ('같이', 58),
 (

In [482]:
rank_text = np.array(rank_text)
rank_text = rank_text[0:,0]

In [483]:
rank_text[1:31]

array(['돈까스', '맛집', '먹스타그램', '돈까스맛집', '00', '수', '맛스타그램', '돈카츠', '돈', '더',
       '일상', '점심', '너무', '11', '있는', '30', '잘', '1', '먹스타', '다', '많이',
       '집밥', '돈가스', '치즈돈까스', '맞팔', '한', '먹고', '정말', '먹방', '진짜'],
      dtype='<U82')

In [ ]:
'떡볶이', '먹스타그램', '순대', '맛집', '일상', '분식', '튀김', '떡볶이맛집', '어묵', '맛스타그램'